In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/'My Drive'/doan

/content/drive/My Drive/doan


**IMPORT DATA**

In [4]:
image_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="training",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "Class_Human",
    validation_split=0.2,
    subset="validation",
    seed=18,
    image_size=image_size,
    batch_size=batch_size,
)


Found 21758 files belonging to 2 classes.
Using 17407 files for training.
Found 21758 files belonging to 2 classes.
Using 4351 files for validation.


In [5]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

**TRAINNING**

Model

In [6]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,concatenate,AveragePooling2D,Rescaling,Conv2D,MaxPooling2D,BatchNormalization,LeakyReLU,GlobalAvgPool2D
from keras.models import Sequential

In [7]:
def conv_block(x, filters, kernel_size, name=None):
  x = Conv2D(filters=filters, kernel_size=kernel_size, padding='same', name=name, use_bias=False)(x)
  x = BatchNormalization(name=name if name is None else '%s_BatchNorm' %name)(x)
  x = LeakyReLU(alpha=0.1, name=name if name is None else '%s_LeakyRelu' %name)(x)
  return x

In [8]:
input_shape = (64, 64, 3)


inputs = tf.keras.Input(shape=input_shape)

x = conv_block(inputs, filters=8, kernel_size=(3,3), name='conv1_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool1_2x2/2')(x)
x = conv_block(x, filters=16, kernel_size=(3,3), name='conv2_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool2_2x2/2')(x)

x = conv_block(x, filters=32, kernel_size=(3,3), name='conv3_3x3')
x = conv_block(x, filters=16, kernel_size=(1,1), name='conv4_1x1')
x = conv_block(x, filters=32, kernel_size=(3,3), name='conv5_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool3_2x2/2')(x)

x = conv_block(x, filters=64, kernel_size=(3,3), name='conv6_3x3')
x = conv_block(x, filters=32, kernel_size=(1,1), name='conv7_1x1')
x = conv_block(x, filters=64, kernel_size=(3,3), name='conv8_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool4_2x2/2')(x)

x = conv_block(x, filters=128, kernel_size=(3,3), name='conv9_3x3')
x = conv_block(x, filters=64, kernel_size=(1,1), name='conv10_1x1')
x = conv_block(x, filters=128, kernel_size=(3,3), name='conv11_3x3')
x = conv_block(x, filters=64, kernel_size=(1,1), name='conv12_1x1')
x = conv_block(x, filters=128, kernel_size=(3,3), name='conv13_3x3')
x = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same', name='maxpool5_2x2/2')(x)

x = conv_block(x, filters=256, kernel_size=(3,3), name='conv14_3x3')
x = conv_block(x, filters=128, kernel_size=(1,1), name='conv15_1x1')
x = conv_block(x, filters=256, kernel_size=(3,3), name='conv16_3x3')
x = conv_block(x, filters=128, kernel_size=(1,1), name='conv17_1x1')
x = conv_block(x, filters=256, kernel_size=(3,3), name='conv18_3x3')

x = conv_block(x, filters=250, kernel_size=(1,1), name='conv19_1x1')
x = GlobalAvgPool2D()(x)
x = Flatten()(x)
outputs = Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv1_3x3 (Conv2D)          (None, 64, 64, 8)         216       
                                                                 
 conv1_3x3_BatchNorm (BatchN  (None, 64, 64, 8)        32        
 ormalization)                                                   
                                                                 
 conv1_3x3_LeakyRelu (LeakyR  (None, 64, 64, 8)        0         
 eLU)                                                            
                                                                 
 maxpool1_2x2/2 (MaxPooling2  (None, 32, 32, 8)        0         
 D)                                                              
                                                             

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard, LearningRateScheduler

In [10]:
loss = tf.keras.losses.BinaryCrossentropy()
callbacks = [
          #Luu bo trong so weight    
          ModelCheckpoint("files/model.h5"),
          #Theo doi loss function va giam LR
          #factor gia tri learning rate se giam 
          #patience: sau so chu ky ma LR se giam
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
          #Truyen lai ket qua vao CSV
          CSVLogger("files/data.csv"),
          TensorBoard(),
          #Theo doi loss function
          #patience: sau so chu ky neu khong cai thien thi dung
          #restore_best_weights: lua chon khoi phuc tu epoch co ket qua tot nhat
          EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
]

In [11]:
model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'])

# Fit model (training)
history = model.fit(train_ds, steps_per_epoch=len(train_ds),
    validation_data=val_ds, validation_steps=len(val_ds), epochs=40, verbose=1,callbacks=callbacks)

Epoch 1/40
544/544 [==============================] - 1216s 2s/step - loss: 0.1365 - accuracy: 0.9457 - val_loss: 0.1486 - val_accuracy: 0.9467 - lr: 0.0010
Epoch 2/40
544/544 [==============================] - 44s 81ms/step - loss: 0.0562 - accuracy: 0.9810 - val_loss: 0.1970 - val_accuracy: 0.9453 - lr: 0.0010
Epoch 3/40
544/544 [==============================] - 45s 82ms/step - loss: 0.0383 - accuracy: 0.9866 - val_loss: 0.0869 - val_accuracy: 0.9729 - lr: 0.0010
Epoch 4/40
544/544 [==============================] - 45s 82ms/step - loss: 0.0308 - accuracy: 0.9894 - val_loss: 0.1211 - val_accuracy: 0.9600 - lr: 0.0010
Epoch 5/40
544/544 [==============================] - 44s 81ms/step - loss: 0.0222 - accuracy: 0.9923 - val_loss: 0.2109 - val_accuracy: 0.9320 - lr: 0.0010
Epoch 6/40
544/544 [==============================] - 44s 81ms/step - loss: 0.0176 - accuracy: 0.9944 - val_loss: 0.0576 - val_accuracy: 0.9809 - lr: 0.0010
Epoch 7/40
544/544 [==============================] - 44s 